In [2]:
using LinearAlgebra,SparseArrays
using DataFrames
using JuMP, Ipopt, Mosek,MosekTools,MathOptInterface

In [3]:
BranchData = DataFrame(From = [ 1, 1, 2, 3, 3, 4], To = [ 2, 3, 4, 4, 5, 5], 
    R = [ 0.0, 0.023, 0.006, 0.020, 0.0, 0.0], X = [ 0.3, 0.145, 0.032, 0.260, 0.320, 0.500], 
    Gsh = [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Bsh = [ 0.0, 0.040, 0.010, 0.0, 0.0, 0.0], 
    T = [ 1.0, 1.0, 1.0, 1.0, 0.98,1], Tl = [1.0, 1.0, 1.0, 1.0, 0.95, 1.0], Tu = [1.0, 1.0, 1.0, 1.0, 1.05, 1.0],
    ϕ = [ 0.0, 0.0, 0.0, -3.0, 0.0, 0.0], ϕl = [ 0.0, 0.0, 0.0, -30.0, 0.0, 0.0], ϕu = [ 0.0, 0.0, 0.0, 30.0, 0.0, 0.0]
    );
ZshB = [ 0.0+0.0*im 0.0+0.3*im 0.05+0.0*im 0.0+0.0*im 0.0+0.0*im]; 
GshB = [ 0.0 0.0 0.05 0.0 0.0];
BshB = [ 0.0 0.3 0.0 0.0 0.0];
BusNum = 5;
nLine= nrow(BranchData);

In [4]:
R = BranchData.R; X = BranchData.X; Bsh = BranchData.Bsh; T = BranchData.T; ϕ = BranchData.ϕ;
Y = conj((R + X*im).^-1); G = real(Y); B = imag(Y);
insertcols!(BranchData, 4, Y = round.(Y,digits = 3));
insertcols!(BranchData, 5, G = round.(G,digits = 3));
insertcols!(BranchData, 6, B = round.(B,digits = 3));
select!(BranchData, Not(:R));
select!(BranchData, Not(:X));
BranchData

,From,To,Y,G,B,Gsh,Bsh,T,Tl
,Int64,Int64,Complex…,Float64,Float64,Float64,Float64,Float64,Float64
1,1,2,0.0+3.333im,0.0,3.333,0.0,0.0,1.0,1.0
2,1,3,1.067+6.727im,1.067,6.727,0.0,0.04,1.0,1.0
3,2,4,5.66+30.189im,5.66,30.189,0.0,0.01,1.0,1.0
4,3,4,0.294+3.824im,0.294,3.824,0.0,0.0,1.0,1.0
5,3,5,0.0+3.125im,0.0,3.125,0.0,0.0,0.98,0.95
6,4,5,0.0+2.0im,0.0,2.0,0.0,0.0,1.0,1.0


In [5]:
Y   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Y,BusNum,BusNum));
G   = real(Y); B = imag(Y);
Gsh = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Gsh,BusNum,BusNum));
Bsh = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Bsh,BusNum,BusNum));

In [6]:
T   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.T,BusNum,BusNum));
ϕ   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.ϕ,BusNum,BusNum));

In [7]:
BusData = DataFrame(Bus = [ 1, 2, 3, 4, 5], 
    PL = [ 0.0, 0.0, 0.0, 0.90, 0.239], QL = [ 0.0, 0.0, 0.0, 0.400, 0.129], 
    Vmin = [ 0.95, 0.95, 0.95, 0.95, 0.95], Vmax = [ 1.05, 1.05, 1.05, 1.05, 1.05]
    );
PL   = BusData.PL;
QL   = BusData.QL;
Vmin = BusData.Vmin;
Vmax = BusData.Vmax;

In [8]:
GenData = DataFrame(Bus = [ 1, 2, 3, 4, 5], 
    PGmin = [ 0.0, 0.0, 0.10, 0.05, 0.0], PGmax = [ 1.0, 0.0, 0.40, 0.40, 0.0], 
    QGmin = [ -0.4, 0.0, -0.2, -0.2, 0.0], QGmax = [ 0.4, 0.0, 0.30, 0.20, 0.0]
    );
PgL = GenData.PGmin;
PgU = GenData.PGmax;
QgL = GenData.QGmin;
QgU = GenData.QGmax;

In [22]:
model = nothing
#model = Model(with_optimizer(Mosek.Optimizer,QUIET=false,MSK_IPAR_INFEAS_REPORT_AUTO=true));
#=model = Model(with_optimizer(Ipopt.Optimizer, linear_solver = "mumps", 
        derivative_test = "second-order",check_derivatives_for_naninf = "yes",
        print_info_string="yes",print_level=1, bound_relax_factor = 0.0))=#
model = Model(with_optimizer(Mosek.Optimizer,QUIET=false,MSK_IPAR_INFEAS_REPORT_AUTO=true))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek

In [23]:
@variable(model, Pg[1:BusNum]);
@variable(model, U[1:BusNum],Bin);
@variable(model, δ[1:BusNum]);
@variable(model, P[1:BusNum,1:BusNum]);

In [24]:
set_start_value(δ[2],deg2rad(-7.5)); set_start_value(δ[3],deg2rad(-4.22)); 
set_start_value(δ[4],deg2rad(-8.20)); set_start_value(δ[5],deg2rad(-8.64));

set_start_value(P[1],0.946); set_start_value(P[3],0.195); set_start_value(P[4],0.058); 


In [25]:
for i in findall(x->x==0, GenData.PGmax)
    fix(Pg[i],0)    
end



for i in 1:BusNum
    set_upper_bound(δ[i], π);
    set_lower_bound(δ[i], -π);
end

In [26]:
@objective(model, Min, 0.35*Pg[1] + 0.2*Pg[3] + 0.4*Pg[3]*Pg[3] + 0.3*Pg[4] + 0.5*Pg[4]*Pg[4])

0.4 Pg[3]² + 0.5 Pg[4]² + 0.35 Pg[1] + 0.2 Pg[3] + 0.3 Pg[4]

In [27]:
@constraint(model, ActivePFij[i = 1:BusNum,j = 1:BusNum], P[i,j] ==
      B[i,j]*(δ[i]-δ[j])  );

In [28]:
@constraint(model, ActiveBranch[i = 1:BusNum],Pg[i]-PL[i] == 
sum(P[i,j] for j in 1:BusNum) );

In [29]:
@constraint(model, MaxPower[i = 1:BusNum],Pg[i] <= PgU[i]*U[i]  );
@constraint(model, MinPower[i = 1:BusNum],Pg[i] >= PgL[i]*U[i] );

In [30]:
optimize!(model);
termination_status(model)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{MosekModel}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities C:\Users\cilli\.julia\packages\MathOptInterface\XE04a\src\Utilities\copy.jl:140


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 44              
  Cones                  : 1               
  Scalar variables       : 45              
  Matrix variables       : 0               
  Integer variables      : 5               

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.00
Presolved problem: 21 variables, 19 constraints, 54 non-zeros
Presolved problem: 0 general integer, 2 binary, 19 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        0        0        NA                   3.8333772211e-01     NA          0.0   
0        1        0        0        3.8458750863e-01     3.8333772211e-01     0.32        0.0   
0        1        0        0        3.8333841891e-01     3.8333772

OPTIMAL::TerminationStatusCode = 1

In [31]:
objective_value(model)

0.38333836688717765

In [47]:
value.(P)

5×5 Array{Float64,2}:
  0.0        0.471904   0.429296  0.0         0.0      
 -0.471904   0.0        0.0       0.471904    0.0      
 -0.429296   0.0        0.0       0.357162    0.259634 
  0.0       -0.471904  -0.357162  0.0        -0.0206344
  0.0        0.0       -0.259634  0.0206344   0.0      

In [ ]:
ϵ = 0.002;
@NLconstraint(model, PFlogic_A[i = 1:BusNum,j = 1:BusNum],ϵ >= P[i,j]^2 - P[j,i]^2 );
@NLconstraint(model, PFlogic_R[i = 1:BusNum,j=BusNum],ϵ >= Q[i,j]^2 - Q[j,i]^2 );